In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
from glob import glob

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import openslide
from openslide import open_slide
from openslide.deepzoom import DeepZoomGenerator
import pandas as pd
from scipy.ndimage.morphology import binary_fill_holes, binary_closing, binary_dilation
import skimage
from skimage.color import rgb2gray
from skimage.morphology import closing, binary_closing, disk, remove_small_holes, dilation, remove_small_objects
from skimage import color, morphology, filters, exposure, feature

plt.rcParams['figure.figsize'] = (10, 6)

# Visualize

In [ ]:
files = glob("data/training_image_data/*.svs")
files

In [ ]:
slide_num = 2
slide = open_slide(files[slide_num-1])

In [ ]:
slide.dimensions

In [ ]:
x, y = slide.dimensions

In [ ]:
x,y

In [ ]:
x*y

In [ ]:
x * y * 3 * 8 / 1024 / 1024 / 1024

In [ ]:
slide.level_count

In [ ]:
slide.level_dimensions

In [ ]:
slide.level_downsamples

In [ ]:
mag = int(slide.properties[openslide.PROPERTY_NAME_OBJECTIVE_POWER])
mag

In [ ]:
slide.get_best_level_for_downsample(20/mag)

In [ ]:
# location = (0, 0)
# level = 2
# size = slide.level_dimensions[level]
# img_whole_slide = slide.read_region(location, level, size)
# img_whole_slide.save("out/images/cancer.jpg")
# img_whole_slide

In [ ]:
# location = (0, 0)
# level = 1
# size = slide.level_dimensions[level]
# img_whole_slide = slide.read_region(location, level, size)
# img_whole_slide.save("out/images/cancer_med_res.jpg")
# img_whole_slide

In [ ]:
# location = (0, 0)
# level = 0
# size = slide.level_dimensions[level]
# img_whole_slide = slide.read_region(location, level, size)
# img_whole_slide.save("out/images/cancer_high_res.jpg")
# img_whole_slide

In [ ]:
location = (40000, 20000) #(13000, 130000)
level = 0
size = (1024, 1024)
img = slide.read_region(location, level, size)
img.save("etc/cancer_zoom.jpg")
img

In [ ]:
img.mode

In [ ]:
img.convert("RGB")

In [ ]:
x = np.array(img)

In [ ]:
x.shape

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(x)
plt.title("Original"); plt.xticks([]); plt.yticks([])
plt.subplot(1, 2, 2)
plt.imshow(x[:, :, :-1])
plt.title("RGB"); plt.xticks([]); plt.yticks([])
plt.show()

In [ ]:
np.r_[0:3, 3:]

In [ ]:
np.r_?

In [ ]:
xt = x.transpose(2, 0, 1)

In [ ]:
xt.shape

# Generate Tiles

In [ ]:
tile_size = 256  #1024
tiles = DeepZoomGenerator(slide, tile_size=tile_size, overlap=0, limit_bounds=False)  # overlap adds pixels to each side

In [ ]:
tiles.level_count

In [ ]:
tiles.tile_count

In [ ]:
tiles.level_tiles

In [ ]:
# Note: Use the magnification property to determine which of these levels to use to get 20x size
tiles.level_dimensions

In [ ]:
tiles.level_dimensions[tiles.level_count-1]

In [ ]:
tile = tiles.get_tile(tiles.level_count-1, (25, 30))
tile

In [ ]:
tile2 = tiles.get_tile(tiles.level_count-1, (25, 41))
tile2

In [ ]:
tile3 = tiles.get_tile(tiles.level_count-1, (40, 30))  # (45, 37))
tile3

In [ ]:
np.array(tile3).shape

In [ ]:
tile4 = tiles.get_tile(tiles.level_count-1, (0, 0))
tile4

In [ ]:
tile5 = tiles.get_tile(tiles.level_count-1, (0, 42))
tile5

In [ ]:
tile6 = tiles.get_tile(tiles.level_count-1, (14, 2))
tile6

In [ ]:
np.array(tile).shape

In [ ]:
gray = rgb2gray(np.array(tile))  #[:, :, np.newaxis]
gray.shape

In [ ]:
gray = gray[:, :, np.newaxis]
gray.shape

In [ ]:
gray.squeeze().shape

In [ ]:
cmap = mpl.colors.ListedColormap(['w', 'k'])  # 0 -> white, 1 -> black
plt.imshow(gray.squeeze(), cmap=cmap, vmin=0, vmax=1)

In [ ]:
type(tile)

In [ ]:
a = [np.array(tile), np.array(tile)]
np.array(a).shape

In [ ]:
a = []
a.append(np.array(tile))
a.append(np.array(tile))
np.array(a).shape

In [ ]:
plt.imshow(np.array(tile))
plt.xticks([]); plt.yticks([])
plt.show()

In [ ]:
(cols, rows) = tiles.level_tiles[-1]
cols, rows

In [ ]:
cols*rows

In [ ]:
500 * 38528

In [ ]:
a = np.array([[1,5,9,13],
              [2,6,10,14],
              [3,7,11,15],
              [4,8,12,16]])
a = np.array([a,a])
a

In [ ]:
a.shape

In [ ]:
a.reshape(a.shape[0], a.shape[1]//2, 2, a.shape[2]//2, 2).swapaxes(2, 3).reshape(-1, 2, 2)

---

# Filter slides with < 90% tissue

In Stanford paper, the following procedure distinguished tissue from background:
1. convert lowest resolution scan of image to grayscale
2. apply automatic contrast enhancement
3. take 8-bit depth complement
4. perform hysteresis thresholding with an “experimentally-chosen high threshold” of 100 and low threshold of 50.

In [ ]:
def calc_tissue_percentage(image):
  """
  Calculates the percentage of the image that is filled by tissue.
  
  Assumes that tissue is white (pixel value = 255), and blank slide is black (0).
  """
  (x, y) = image.shape
  percentage = image.sum() / (x * y)# * 255)
  return percentage

In [ ]:
import ftdetect
from ftdetect import cleanedges
cleanedges.hystThresh?

In [ ]:
def normalize_staining(x, beta=0.15, alpha=1, light_intensity=240):
  """
  Normalize the staining of H&E histology slides.
  
  This function normalizes the staining of H&E histoloy slides
  using the method described in [1].
  """
  # Setup.
  x = np.asarray(x)
  h, w, c = x.shape
  x = x.reshape(-1, c).astype(np.float64)  # shape (H*W, C)
  
  # Reference stain vectors and stain saturations.  We will normalize all slides
  # to these references.  To create these, grab the stain vectors and stain
  # saturations from a desirable slide.
  ## Values in reference implementation for use with eigendecomposition approach.
#   stain_ref = np.array([0.5626, 0.2159, 0.7201, 0.8012, 0.4062, 0.5581]).reshape(3,2)
#   max_sat_ref = np.array([1.9705, 1.0308]).reshape(2,1)
  ## Values for use with SVD approach.  These were computed by (1) running the
  ## the eigendecomposition approach to normalize an image, (2) running the
  ## SVD approach on the normalized image, and (3) recording the stain vectors
  ## and max saturations for this (ideal) normalized image.
  stain_ref = np.array([0.20730702, 0.56170196, 0.80308092, 0.72012455, 0.55864554, 0.4073224]).reshape(3,2)
  max_sat_ref = np.array([0.99818645, 1.96029115]).reshape(2,1)
  
  # Convert RGB to OD.
  OD = -np.log((x+1)/light_intensity)  # shape (H*W, C)
#   OD = -np.log10(x/255 + 1e-8)
  
  # Remove data with OD intensity less than beta.
  # I.e. remove transparent pixels.
  # Note: This needs to be checked per channel, rather than
  # taking an average over all channels for a given pixel.
  #OD_thresh = OD[np.logical_not(np.any(OD < beta, 1)), :]
  OD_thresh = OD[np.all(OD >= beta, 1), :]  # shape (K, C)
  
  # Calculate eigenvectors.
#   eigvals, eigvecs = np.linalg.eig(np.cov(OD_thresh.T))  # np.cov results in inf/nans
  U, s, V = np.linalg.svd(OD_thresh, full_matrices=False)
  
  # Extract two largest eigenvectors.
  # Note: We swap the sign of the eigvecs here to be consistent
  # with other implementations.  Both +/- eigvecs are valid, with
  # the same eigenvalue, so this is okay.
#   top_eigvecs = eigvecs[:, np.argsort(eigvals)[-2:]] * -1
  top_eigvecs = V[0:2, :].T * -1  # shape (C, 2)
  
  # Project thresholded optical density values onto plane spanned by
  # 2 largest eigenvectors.
  proj = np.dot(OD_thresh, top_eigvecs)  # shape (K, 2)
  
  # Calculate angle of each point wrt the first plane direction.
  # Note: the parameters are `np.arctan2(y, x)`
  angles = np.arctan2(proj[:, 1], proj[:, 0])  # shape (K,)
  
  # Find robust extremes (a and 100-a percentiles) of the angle.
  min_angle = np.percentile(angles, alpha)
  max_angle = np.percentile(angles, 100-alpha)
  
  # Convert min/max vectors (extremes) back to OD space.
  extreme_angles = np.array(
    [np.cos(min_angle), np.cos(max_angle), np.sin(min_angle), np.sin(max_angle)]
  ).reshape(2,2)
  stains = np.dot(top_eigvecs, extreme_angles)  # shape (C, 2)
#   min_vec = np.dot(top_eigvecs, np.array([np.cos(min_angle), np.sin(min_angle)]).reshape(2,1))
#   max_vec = np.dot(top_eigvecs, np.array([np.cos(max_angle), np.sin(max_angle)]).reshape(2,1))
  
#   # Merge vectors with hematoxylin first, and eosin second with a heuristic.
#   if min_vec[0] > max_vec[0]:
#     stains = np.hstack((min_vec, max_vec))
#   else:
#     stains = np.hstack((max_vec, min_vec))

  # Calculate saturations of each stain.
  # Note: Here, we solve
  #    OD = VS
  #     S = V^{-1}OD
  # where `OD` is the matrix of optical density values of our image,
  # `V` is the matrix of stain vectors, and `S` is the matrix of stain
  # saturations.  Since this is an overdetermined system, we use the
  # least squares solver, rather than a direct solve.
  sats, _, _, _ = np.linalg.lstsq(stains, OD.T)
  
  # Normalize stain saturations.
  max_sat = np.percentile(sats, 99, axis=1, keepdims=True)
  sats = sats / max_sat * max_sat_ref
  
  # Recreate image.
  # Note: If the image is immediately converted to uint8 with `.astype(np.uint8)`, it will
  # not return the correct values due to the initital values being outside of [0,255].
  # To fix this, we round to the nearest integer, and then clip to [0,255], which is the
  # same behavior as Matlab.
  x_norm = np.exp(np.dot(-stain_ref, sats)) * light_intensity
#   x_norm = 10**(np.dot(-stain_ref, sats)) * 255
  x_norm = np.clip(np.round(x_norm), 0, 255).astype(np.uint8)
  x_norm = x_norm.T.reshape(h,w,c)
  
  # Debug.
#   print("OD shape: ", OD.shape)
#   print("OD_thresh shape: ", OD_thresh.shape)
#   print("eigvals: ", eigvals)
#   print("sorted eigvals: ", np.argsort(eigvals))
#   print("top_eigvecs shape: ", top_eigvecs.shape)
#   print("top_eigvecs: ", top_eigvecs)
#   print("top 2 eigval indices: ", np.argsort(eigvals)[-2:])
#   print("proj shape: ", proj.shape)
#   print("proj mean: ", np.mean(proj, axis=0))
#   print("angles shape: ", angles.shape)
#   print("angles mean: ", np.mean(angles))
#   print("min/max angles: ", min_angle, max_angle)
#   print("min_vec shape: ", min_vec.shape)
#   print("min_vec mean: ", np.mean(min_vec))
#   print("max_vec mean: ", np.mean(max_vec))
#   print("stains shape: ", stains.shape)
  print("stains: ", stains)
#   print("sats shape: ", sats.shape)
#   print("sats mean: ", np.mean(sats, axis=1))
#   print("max_sat shape: ", max_sat.shape)
  print("max_sat: ", max_sat)
#   print("x_norm shape: ", x_norm.shape)
#   print("x_norm mean: ", np.mean(x_norm, axis=(0,1)))
#   print("x_norm min: ", np.min(x_norm, axis=(0,1)))
#   print("x_norm max: ", np.max(x_norm, axis=(0,1)))
#   print(x_norm.dtype)
#   print()
# #   x = x.reshape(h,w,c).astype(np.uint8)
  
  return x_norm

In [ ]:
rgb2gray(np.asarray(tile)).dtype

In [ ]:
def optical_density(tile):
  """
  Convert a tile to optical density values.
  
  Args:
    tile: A 3D NumPy array of shape (tile_size, tile_size, channels).
  
  Returns:
    A 3D NumPy array of shape (tile_size, tile_size, channels) representing
    optical density values.
  """
  tile = tile.astype(np.float64)
  #od = -np.log10(tile/light_intensity + 1e-8)
  od = -np.log((tile+1)/240)
  return od

def process_tile(tile):
  # Convert to numpy array
  image = np.array(tile)

#   # 1. convert to grayscale
#   im = np.round(color.rgb2gray(image) * 255).astype(np.uint8)  # 0 -> black, 255 -> white

#   # 2. contrast enhancement
#   im = filters.rank.enhance_contrast(im, disk(3))

#   # 3. 8-bit depth complement
#   im = 255 - im  # 0 -> white, 255 -> black

#   # 4. Hysteresis thresholding (not finished)
#   im_thresh = im / 255
#   im_thresh[im < 50] = 0
#   im_thresh[im > 100] = 1
#   im_transf = im_thresh
#   im = im / 255

#   # Binary threshold
# #   im_thresh = (im * 255 > 30)  # 0 -> white, 1 -> black
#   od = optical_density(im)
#   im_thresh = od > 0.15  #0.2

#   # Morphology to remove noise
#   im_transf = im_thresh
#   im_transf = morphology.binary_closing(im_thresh, disk(10))
# #   im_transf = morphology.remove_small_holes(im_transf, 100)
#   im_transf = dilation(im_transf, disk(10))
#   im_transf = binary_fill_holes(im_transf)
# #   im_transf = morphology.remove_small_objects(im_transf)


#   im = 1 - rgb2gray(image)
#   im_thresh = feature.canny(im)
#   im_transf = im_thresh
#   im_transf = morphology.binary_closing(im_thresh, disk(10))
#   im_transf = morphology.binary_dilation(im_transf, disk(10))
#   im_transf = binary_fill_holes(im_transf)
  
  im = optical_density(image)
  im_thresh = np.min(im, axis=2) >= 0.15
  im_transf = morphology.binary_closing(im_thresh, disk(2))
  im_transf = morphology.binary_dilation(im_transf, disk(2))
  im_transf = binary_fill_holes(im_transf)

#   im = image
#   im = normalize_staining(image)
#   im = 1 - rgb2gray(im)
# #   im_thresh = im
#   im_thresh = cleanedges.hystThresh(im, 50/255, 100/255)
#   im_transf = im_thresh
#   im_transf = morphology.binary_closing(im_thresh, disk(1))
# #   im_transf = morphology.binary_dilation(im_transf, disk(1))
# #   im_transf = binary_fill_holes(im_transf)

  # plot
  cmap = mpl.colors.ListedColormap(['w', 'k'])  # 0 -> white, 1 -> black
  fig = plt.figure(figsize=(10, 10))
  plt.subplot(2,2,1)
  plt.imshow(image), plt.xticks([]), plt.yticks([])
  plt.subplot(2,2,2)
  plt.imshow(im, cmap=cmap, vmin=0, vmax=1), plt.xticks([]), plt.yticks([])
  plt.subplot(2,2,3)
  plt.imshow(im_thresh, cmap=cmap, vmin=0, vmax=1), plt.xticks([]), plt.yticks([])
  plt.subplot(2,2,4)
  plt.imshow(im_transf, cmap=cmap, vmin=0, vmax=1), plt.xticks([]), plt.yticks([])
  plt.show()

  print(im_thresh.mean())
  print(im_transf.mean())

  return im_transf, im_thresh, im

In [ ]:
def keep_tile(tile, tile_size, tissue_threshold=0.9):
  if tile.shape[0:2] == (tile_size, tile_size):
    tile_orig = tile
    
    # Check 1
    # Convert 3D RGB image to 2D grayscale image, from
    # 0 (dense tissue) to 1 (plain background).
    tile = rgb2gray(tile)
    # 8-bit depth complement, from 1 (dense tissue)
    # to 0 (plain background).
    tile = 1 - tile
    # Canny edge detection with hysteresis thresholding.
    # This returns a binary map of edges, with 1 equal to
    # an edge. The idea is that tissue would be full of
    # edges, while background would not.
    tile = canny(tile)
    # Binary closing, which is a dilation followed by
    # an erosion. This removes small dark spots, which
    # helps remove noise in the background.
    tile = binary_closing(tile, disk(10))
    # Binary dilation, which enlarges bright areas,
    # and shrinks dark areas. This helps fill in holes
    # within regions of tissue.
    tile = binary_dilation(tile, disk(10))
    # Fill remaining holes within regions of tissue.
    tile = binary_fill_holes(tile)
    # Calculate percentage of tissue coverage.
    percentage = tile.mean()
    print(percentage)
    check1 = percentage >= tissue_threshold
    
    # Check 2
    beta = 0.15
    tile = optical_density(tile_orig)
    tile = np.min(tile, axis=2) >= beta
    tile = binary_closing(tile, disk(2))
    tile = binary_dilation(tile, disk(2))
    tile = binary_fill_holes(tile)
    percentage = tile.mean()
    print(percentage)
    check2 = percentage >= tissue_threshold
    
    return check1 and check2
  else:
    return False

In [ ]:
im1 = process_tile(tile)

In [ ]:
keep_tile(np.array(tile), tile_size)

In [ ]:
im2 = process_tile(tile2)

In [ ]:
keep_tile(np.array(tile2), tile_size)

In [ ]:
im3 = process_tile(tile3)

In [ ]:
keep_tile(np.array(tile3), tile_size)

In [ ]:
im4, im4_thresh, im4_contrast = process_tile(tile4)

In [ ]:
keep_tile(np.array(tile4), tile_size)

In [ ]:
im5 = process_tile(tile5)

In [ ]:
keep_tile(np.array(tile5), tile_size)

In [ ]:
im6 = process_tile(tile6)

In [ ]:
keep_tile(np.array(tile6), tile_size)

In [ ]:
# Hysteresis thresholding experiments

from skimage import measure
import scipy.ndimage as ndi
from scipy.ndimage import (gaussian_filter,
                           generate_binary_structure, binary_erosion, label)
image = np.array(tile3)

# 1. convert to grayscale
im = np.round(color.rgb2gray(image) * 255).astype(np.uint8)  # 0 -> black, 255 -> white

# 2. contrast enhancement
im = filters.rank.enhance_contrast(im, disk(3))

# 3. 8-bit depth complement
im = 255 - im  # 0 -> white, 255 -> black

# 4. Hysteresis thresholding
low_mask = im > 50
high_mask = im > 100

# low_labels = measure.label(low_mask, background=0)
# high_labels = measure.label(high_mask, background=0)
# labels = low_labels * high_labels

# https://github.com/scikit-image/scikit-image/blob/master/skimage/feature/_canny.py#L53
output_mask = low_mask
strel = np.ones((3, 3), bool)
labels, count = label(low_mask, strel)
print(labels, count)
if count > 0:
  sums = (np.array(ndi.sum(high_mask, labels,
                           np.arange(count, dtype=np.int32) + 1),
                   copy=False, ndmin=1))
  good_label = np.zeros((count + 1,), bool)
  good_label[1:] = sums > 0
  output_mask = good_label[labels]

cmap = mpl.colors.ListedColormap(['w', 'k'])  # 0 -> white, 1 -> black
fig = plt.figure(figsize=(20, 20))
plt.subplot(3,3,1)
plt.imshow(image), plt.xticks([]), plt.yticks([])
plt.subplot(3,3,2)
plt.imshow(im, cmap='gray'), plt.xticks([]), plt.yticks([])
plt.subplot(3,3,4)
plt.imshow(low_mask, cmap=cmap, vmin=0, vmax=1), plt.xticks([]), plt.yticks([])
plt.subplot(3,3,5)
plt.imshow(high_mask, cmap=cmap, vmin=0, vmax=1), plt.xticks([]), plt.yticks([])
plt.subplot(3,3,7)
plt.imshow(output_mask, cmap=cmap), plt.xticks([]), plt.yticks([])
plt.subplot(3,3,8)
plt.imshow(output_mask-low_mask, cmap=cmap), plt.xticks([]), plt.yticks([])
# plt.subplot(3,3,9)
# plt.imshow(labels, cmap='spectral'), plt.xticks([]), plt.yticks([])
plt.show()
print(output_mask.mean())

In [ ]:
measure.label?

In [ ]:
print(np.array(tile4).mean(), im4_contrast.mean())

In [ ]:
im4.shape

In [ ]:
np.array(tile).max()

In [ ]:
rgb2gray(np.array(tile)).max()

In [ ]:
i = np.array(tile)

In [ ]:
i.dtype

In [ ]:
i.max()

In [ ]:
ig = rgb2gray(i)

In [ ]:
ig.dtype

In [ ]:
ig.max()

In [ ]:
skimage.color.gray2rgb(ig).shape

In [ ]:
igi = np.round(ig * 255).astype("uint8")

In [ ]:
igi.dtype

In [ ]:
igi.max()

In [ ]:
np.allclose(ig, igi/255)

In [ ]:
ig

In [ ]:
igi/255

In [ ]:
plt.imshow(ig, cmap="gray")

In [ ]:
plt.imshow(igi/255, cmap="gray")

In [ ]:
tile

In [ ]:
new_size = 64
tile = np.array(tile)
x, y, ch = tile.shape
samples = (tile.reshape((x // new_size, new_size, y // new_size, new_size, ch))
               .swapaxes(1,2)
               .reshape((-1, new_size, new_size, ch))
               .transpose(0,3,1,2))
samples = samples.reshape(samples.shape[0], -1)

In [ ]:
def visualize_sample(sample, size=256):
  """
  Plot a tissue sample.
  
  Args:
    sample: A square sample flattened to a vector of size
      (channels*size_x*size_y).
    size: The width and height of the square samples.
  
  Returns:
    None
  """
  # Change type, reshape, transpose to (size_x, size_y, channels).
  length = sample.shape[0]
  channels = int(length / (size * size))
  if channels > 1:
    sample = sample.astype('uint8').reshape((channels, size, size)).transpose(1,2,0)
    plt.imshow(sample)
  else:
    vmax = 255 if sample.max() > 1 else 1
    sample = sample.reshape((size, size))
    plt.imshow(sample, cmap="gray", vmin=0, vmax=vmax)
  plt.show()

In [ ]:
len(samples)

In [ ]:
visualize_sample(samples[24], new_size)

---

# Other Stuff

In [ ]:
import skimage

In [ ]:
image = np.array(tile6)
image_gray = rgb2gray(image)
edges = feature.canny(image_gray)
edges2 = feature.canny(image_gray, sigma=3)
edges3 = feature.canny(image_gray, low_threshold=0.15)
close = morphology.binary_closing(edges, disk(10))

cmap = mpl.colors.ListedColormap(['w', 'k'])  # 0 -> white, 1 -> black
plt.subplot(2,3,1)
plt.imshow(image)
plt.subplot(2,3,2)
plt.imshow(image_gray, interpolation='nearest', cmap=cmap, vmin=0, vmax=1)
plt.subplot(2,3,3)
plt.imshow(edges, interpolation='nearest', cmap=cmap, vmin=0, vmax=1)
plt.subplot(2,3,4)
plt.imshow(edges2, interpolation='nearest', cmap=cmap, vmin=0, vmax=1)
plt.subplot(2,3,5)
plt.imshow(edges3, interpolation='nearest', cmap=cmap, vmin=0, vmax=1)
plt.subplot(2,3,6)
plt.imshow(close, interpolation='nearest', cmap=cmap, vmin=0, vmax=1)

plt.show()

print(calc_tissue_percentage(edges), calc_tissue_percentage(edges2), calc_tissue_percentage(close))

In [ ]:
image = np.array(tile4)

# im = skimage.img_as_ubyte(color.rgb2gray(image)) #* 255  # 0 -> black, 255 -> white
# im = color.rgb2hed(image)
im = color.rgb2gray(image)
# im = filters.rank.enhance_contrast(im, disk(3))
# im = 255 - im
# im = im > 50
# print(calc_tissue_percentage(im))
im

In [ ]:
image = np.array(tile4)
image = skimage.img_as_float(image)
black_mask = color.rgb2gray(image) < 0.1
distance_red = color.rgb2gray(1 - np.abs(image - (1,0,0)))
distance_blue = color.rgb2gray(1 - np.abs(image - (0,0,1)))

distance_red[black_mask] = 0
distance_blue[black_mask] = 0

plt.subplot(1,3,1)
plt.imshow(image)
plt.subplot(1,3,2)
plt.imshow(distance_red, interpolation='nearest', cmap='gray')
plt.subplot(1,3,3)
plt.imshow(distance_blue, interpolation='nearest', cmap='gray')
plt.show()

In [ ]:
image = np.array(tile4)
# image = skimage.img_as_float(image)
# image = color.rgb2gray(image)
# image = filters.rank.enhance_contrast(skimage.img_as_ubyte(image), disk(5))
lowpass = filters.gaussian(image, 100)
highpass = image - lowpass
highpass -= highpass.min()

plt.subplot(1,3,1)
plt.imshow(image) #, cmap='gray')
plt.subplot(1,3,2)
plt.imshow(lowpass) #, cmap='gray')
plt.subplot(1,3,3)
plt.imshow(highpass) #, cmap='gray')

plt.show()

In [ ]:
def optical_density(image):
  od = -np.log10(image/255)  # optical density of each channel
  od = np.average(od, axis=2)  # optical density averaged over channels
  return od

image = np.array(tile4)
od = optical_density(image)
thresh = od > 0.2

plt.subplot(1,3,1)
plt.imshow(image) #, cmap='gray')
plt.subplot(1,3,2)
plt.imshow(od, cmap='gray')
plt.subplot(1,3,3)
plt.imshow(thresh, cmap='gray')

plt.show()

In [ ]:
thresh.shape

In [ ]:
from skimage.filters import sobel
from skimage.morphology import watershed

test_im = rgb2gray(np.array(tile2)) * 255
elevation_map = sobel(test_im)
markers = np.zeros_like(test_im)
markers[test_im < 50] = 1
markers[test_im > 100] = 2
segmentation = watershed(elevation_map, markers)  - 1
# segmentation = binary_fill_holes(segmentation - 1)
plt.subplot(1,3,1)
plt.imshow(markers)
plt.subplot(1,3,2)
plt.imshow(elevation_map)
plt.subplot(1,3,3)
plt.imshow(segmentation)
plt.show()

In [ ]:
from skimage import data
from skimage.color import separate_stains, hdx_from_rgb, hed_from_rgb
ihc = data.immunohistochemistry()
ihc_hdx = separate_stains(ihc, hdx_from_rgb)

In [ ]:
plt.subplot(1,2,1)
plt.imshow(ihc)
plt.subplot(1,2,2)
plt.imshow(ihc_hdx)
plt.show()

In [ ]:
image = np.array(tile4)
image = color.rgb2gray(image)
image = image - image.mean()
# image = (image < filters.threshold_otsu(image))
# image = (image < filters.threshold_adaptive(image, 21))
image = (image < filters.threshold_li(image))
plt.imshow(1-image, cmap="gray")

In [ ]:
tile_orig = tile
tile_hed = separate_stains(tile_orig, hed_from_rgb)

fig, axes = plt.subplots(2, 2, figsize=(12, 9), sharex=True, sharey=True, subplot_kw={'adjustable':'box-forced'})
ax0, ax1, ax2, ax3 = axes.ravel()

ax0.imshow(tile_orig)
ax0.set_title("Original image")

ax1.imshow(tile_hed[:, :, 0], cmap=plt.cm.gray)
ax1.set_title("Hematoxylin")

ax2.imshow(tile_hed[:, :, 1], cmap=plt.cm.gray)
ax2.set_title("Eosin")

ax3.imshow(tile_hed[:, :, 2], cmap=plt.cm.gray)
ax3.set_title("DAB")

for ax in axes.ravel():
    ax.axis('off')

fig.subplots_adjust(hspace=0.3)

plt.show()

In [ ]:
from skimage.exposure import rescale_intensity

# Rescale hematoxylin and DAB signals and give them a fluorescence look
h = rescale_intensity(tile_hed[:, :, 0], out_range=(0, 1))
d = rescale_intensity(tile_hed[:, :, 2], out_range=(0, 1))
zdh = np.dstack((np.zeros_like(h), d, h))

#fig, ax = plt.subplots()
fig = plt.figure()
ax = plt.subplot(1, 1, 1, sharex=ax0, sharey=ax0, adjustable='box-forced')
ax.imshow(zdh)
ax.set_title("Stain separated image (rescaled)")
ax.axis('off')
plt.show()

In [ ]:
from skimage import io, filters, color, exposure, morphology, feature, draw, measure, transform

In [ ]:
A = np.array(tile2)
io.imshow(A)
plt.show()

In [ ]:
image = exposure.equalize_adapthist(A)
io.imshow(image)
plt.show()

In [ ]:
binary = filters.threshold_adaptive(exposure.adjust_sigmoid(A[:, :, 0], cutoff=0.4, gain = 30), 301).astype(bool)
clean = morphology.binary_closing(binary, morphology.disk(3)).astype(bool)
clean = morphology.remove_small_objects(clean, 200)
clean = morphology.remove_small_objects( (1-clean).astype(bool), 200)

io.imshow(clean)

In [ ]:
import matplotlib
# Find contour of inflammatory zone

local_density = filters.gaussian(clean, 61)
local_density -= local_density.min()
local_density /= local_density.max()

ent = filters.gaussian(filters.rank.entropy(local_density, morphology.disk(3)), 75)
ent -= ent.min()
ent /= ent.max()

info = ent * (1 + local_density)

bw = (info) > filters.threshold_otsu(info)

C = measure.find_contours(bw, 0.5)
centroid = []
vals = []

for c in C :
    centroid.append(np.linalg.norm([c[:, 1].mean() - bw.shape[1] / 2, c[:, 0].mean() - bw.shape[0] / 2]))
    vals.append(local_density.T[c.astype(int)].sum())

cent = C[np.argmin(centroid / np.array(vals))]
path = matplotlib.path.Path(cent)

io.imshow(image)
plt.plot(cent[:, 1], cent[:, 0], lw=5, c="k", alpha = 0.7)
plt.grid(False)